In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 

import utils.tavr_torch as tavr_torch
from utils.tavr_torch import TAVR_3_Frame, TAVR_1_Frame, TAVR_Sequence, tavr_dataloader
from utils.visualization import display_grid, z_stretch, visualize_frame, set_figsize, get_central_slices
from utils.loss_functions import batch_l2_loss
from utils.run_model import train, test, save, load, get_loss_history
from Models.basic_models import average_model, two_layer_basic, post_process, two_d_basic

set_figsize(20, 15)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
USE_GPU = False
dtype = torch.float32 # we will be using float throughout this tutorial
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print('using device:', device)

# "Pixl", "Slice", or "None"
preproc_type = "pixel"

validation = TAVR_3_Frame("__valid", preproc=preproc_type)
val_loader = tavr_dataloader(validation, batch_size=4, shuffle=True, num_workers=2)
training = TAVR_3_Frame("__train", preproc=preproc_type)
train_loader = tavr_dataloader(training,batch_size=8, shuffle=True, num_workers=2)


ave_model = average_model()
model = two_d_basic()
post_proc = post_process(kind=preproc_type).to(device=device)
loss_fn = batch_l2_loss()

# CHANGE TO NAME OF JUPYTER NOTEBOOK
model_name = "Model 6 (2d CNN+PixelNorm-Adam) Run 1"

using device: cpu


In [5]:
learning_rate = 3e-3
momentum = 0.90
reg = 1e-7

optimizer = optim.Adam(model.parameters())

In [6]:
# Run cell to load model
LOAD = False
iteration_num = -1

if LOAD:
    
    load(model_name, iteration_num, model, optimizer)
    loss_history = get_loss_history(model_name)
    model.to(device=device)
else:
    loss_history = None

In [ ]:
train(model, post_proc, optimizer, train_loader, val_loader, loss_fn, device, 
         model_name, loss_history, epochs=3, print_every=30, print_level=3, lr_decay=1)


Iteration 0, loss = 2.4332, corrected loss = 1132.2378
Validation loss 1035.4302 over 81 frames
model saved to model_checkpoints/Model 6 (2d CNN+PixelNorm-Adam) Run 1/Model 6 (2d CNN+PixelNorm-Adam) Run 1-0
conv_a1.weight,   	norm: 4.7234e+00, 	update norm: 8.4490e-03 	Update/norm: 1.7888e-03
conv_a1.bias,   	norm: 5.6396e-01, 	update norm: 2.8053e-03 	Update/norm: 4.9743e-03
conv_b1.weight,   	norm: 3.5765e+00, 	update norm: 8.4520e-03 	Update/norm: 2.3632e-03
conv_b1.bias,   	norm: 6.7347e-01, 	update norm: 2.8205e-03 	Update/norm: 4.1879e-03
final.weight,   	norm: 1.5431e+00, 	update norm: 3.8529e-03 	Update/norm: 2.4968e-03
final.bias,   	norm: 2.1620e-01, 	update norm: 1.0013e-03 	Update/norm: 4.6312e-03

... ... ... 
Iter 10... ... ... 
Iter 20... ... ... 
Iteration 30, loss = 1.6144, corrected loss = 751.2124
Validation loss 685.0303 over 81 frames
model saved to model_checkpoints/Model 6 (2d CNN+PixelNorm-Adam) Run 1/Model 6 (2d CNN+PixelNorm-Adam) Run 1-30
conv_a1.weight,   	

In [ ]:
# One last test + visualize results on 1 validation sequence
val_seq = TAVR_Sequence("__valid")
test(model, val_loader, loss_fn, device)
test(ave_model, val_loader, loss_fn, device)

with torch.no_grad():
    model.eval()
    ave_frames = [val_seq[0][0]]
    for i in range(2,9,2):
        ave_frame = model((val_seq[0][i-2][None,:], val_seq[0][i][None,:]))
        ave_frames += [ave_frame[0][0],  val_seq[0][i]]
    ave_frames += [val_seq[0][9]]
    ave_frames_slices = []
    for f in ave_frames:
        ave_frames_slices += get_central_slices(f)
    set_figsize(6,20)
    display_grid(10, 3, ave_frames_slices)

In [ ]:
optimizer.param_groups[0]['lr']

In [ ]:
for p in model.parameters():
    print(p.norm())